# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,10.39,50,89,2.10,CL,1698669041
1,1,ust-nera,64.5667,143.2000,-16.73,97,74,1.06,RU,1698669087
2,2,college,64.8569,-147.8028,-3.03,86,75,0.00,US,1698669010
3,3,port-aux-francais,-49.3500,70.2167,2.47,64,71,14.37,TF,1698669019
4,4,uglich,57.5275,38.3317,-1.55,95,100,4.05,RU,1698669458


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    hover_cols=["City"],
    line_color="black", 
    color = "City" 
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

#Filter ideal conditions: Max temperature less than 27 degrees Celsius, Max temperature greater than 21 degrees Celsius,
#0% cloudiness, Wind speed less than 4.5 m/s

indeal_conditions_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Cloudiness']== 0) & (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
clean_data_df = indeal_conditions_df.dropna()

# Display sample data
clean_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
157,157,cabo san lucas,22.8909,-109.9124,24.17,84,0,0.45,MX,1698668870
187,187,hazro city,33.9091,72.4921,25.35,34,0,1.53,PK,1698669494
319,319,dobrota,42.4542,18.7683,23.99,53,0,2.06,ME,1698669523
349,349,gohpur,26.8833,93.6333,22.59,79,0,0.69,IN,1698669528
438,438,clearwater,27.9659,-82.8001,21.71,84,0,0.89,US,1698669543
511,511,haiku-pauwela,20.9219,-156.3051,24.47,81,0,1.54,US,1698669173
578,578,kailua-kona,19.6406,-155.9956,24.77,79,0,3.13,US,1698669044
586,586,katsuren-haebaru,26.3369,127.8719,24.01,73,0,2.57,JP,1698669583


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
157,cabo san lucas,MX,22.8909,-109.9124,84,
187,hazro city,PK,33.9091,72.4921,34,
319,dobrota,ME,42.4542,18.7683,53,
349,gohpur,IN,26.8833,93.6333,79,
438,clearwater,US,27.9659,-82.8001,84,
511,haiku-pauwela,US,20.9219,-156.3051,81,
578,kailua-kona,US,19.6406,-155.9956,79,
586,katsuren-haebaru,JP,26.3369,127.8719,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel", 
    "limit":20,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
hazro city - nearest hotel: No hotel found
dobrota - nearest hotel: No hotel found
gohpur - nearest hotel: No hotel found
clearwater - nearest hotel: The Fort Harrison Hotel
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kailua-kona - nearest hotel: Kona Seaside Hotel
katsuren-haebaru - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
157,cabo san lucas,MX,22.8909,-109.9124,84,Comfort Rooms
187,hazro city,PK,33.9091,72.4921,34,No hotel found
319,dobrota,ME,42.4542,18.7683,53,No hotel found
349,gohpur,IN,26.8833,93.6333,79,No hotel found
438,clearwater,US,27.9659,-82.8001,84,The Fort Harrison Hotel
511,haiku-pauwela,US,20.9219,-156.3051,81,Inn At Mama's Fish House
578,kailua-kona,US,19.6406,-155.9956,79,Kona Seaside Hotel
586,katsuren-haebaru,JP,26.3369,127.8719,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    hover_cols=["City","Hotel Name","Country"],
    line_color="black", 
    color = "City"
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)